# Parte 3: Preguntas y Respuestas en Documentos

## Preprocesamiento

Para el preprocesamiento haremos las siguientes cosas:

1. Pasar preguntas y contextos a arreglos de palabras.
2. A partir campo `answer_start` crear un indice de palabra y no de caracter `answer_word_start`.
3. Agregar campo `answer_word_end`

Guardaremos el archivo en un JSON nuevo para no tener que repetir el preprocesamiento.

In [ ]:
# Importar dependencias para el preprocesamiento.
from nltk.tokenize import word_tokenize
import string
import re
import numpy as np
import json

In [ ]:
with open("train-v1.1.json", "r") as data:
    train = json.load(data)['data']
with open("dev-v1.1.json", "r") as data:
    test = json.load(data)['data']

In [ ]:
# Funciones de preprocesamiento

def preprocess_dataset(data):
    for document in data:
        for paragraph in document['paragraphs']:
            preprocess_paragraph(paragraph)

def preprocess_paragraph(paragraph):
    # preprocesamos contexto
    preprocess_context(paragraph)
    for question in paragraph['qas']:
        # preprocesamos preguntas.
        preprocess_question(paragraph['context'],question)

def preprocess_context(paragraph):

    # Guardamos contexto como arreglo preprocesado
    paragraph['context_tokenized'] =  preprocess_text(paragraph['context'])

def preprocess_question(context,question):
    
    # guardamos pregunta como arreglo
    question['question_tokenized'] = preprocess_text(question['question'])
    for answer in question['answers']:
        # preprocesamos respuestas
        preprocess_answer(context, answer)
    
def preprocess_answer(context,answer):
    
    # Pasamos respuesta a arreglo
    answer['text_tokenized'] = preprocess_text(answer['text'])
    

    # Contamos cantidad de palabras hasta la respuesta
    answer_word = len(preprocess_text(context[:answer['answer_start']]))
    
    # Guardamos en el hash
    answer['answer_word_start'] = answer_word
    
    # Guardamos la palabra final de la respusta en el hash.
    answer['answer_word_end'] = answer_word + len(answer['text_tokenized']) - 1
    
"""
retorna string tokenizado y limpio de simbolos y mayusculas. Esto ultimo es necesario para disminuir
las veces en que GLove no tiene la palabra.
"""
def preprocess_text(text):
    result = text
    
    # minusculas
    result = result.lower()
    
    # simbolos para eliminar
    symbols = re.sub("[{}]".format(string.ascii_letters + "'1234567890" ),"",string.printable)
    
    # eliminamos simbolos mediante regexp.
    result = re.sub("[{}–]".format(symbols)," ", result)
    
    return word_tokenize(result)


In [ ]:
# Preprocesamos y guardamos el nuevo dataset
with open("train-v1.1-pr.json","w") as out:
    print("preprocessing training set")
    preprocess_dataset(train)
    json.dump(train,out)
    
with open("dev-v1.1-pr.json","w") as out:
    print("preprocessing test set")
    preprocess_dataset(test)
    json.dump(test,out)

## Embeddings

Para usar los embeddings e inyectarlos en el modelo de Keras primero se intento lo siguiente:

1. Construiremos un index de palabras a partir del vocabulario del dataset.
2. Transformamos las secuencias de palabras en secuencias de enteros mediante el index.
3. Estandarizamos el Tamaño de la secuencia usando `pad_sequences` de Keras.
4. Luego  construimos matriz de pesos a partir de Glove para inyectar a una capa `Embedding` de Keras.

Este proceso no funcionó porque los vectores de glove ocupaban demasiada memoria de la gpu (eran 50M de parametros), a cambio se optó por precomputar los vectores de cada palabra y pasar los tensores de una sequencia directamente a la red. Como el dataset crece demasiado al pasar cada palabra a un vector de 300D, esta transformación se genera de a poco mediante un objeto Keras Sequence ( se probó primero un generador pero no era compatible con el paralelismo).

In [3]:
# Importamos dependencias para generar los embeddings
from gensim.models import KeyedVectors
from gensim.test.utils import datapath, get_tmpfile
from gensim.scripts.glove2word2vec import glove2word2vec
from keras.preprocessing.sequence import pad_sequences
from keras.utils.np_utils import to_categorical
from keras.utils import Sequence
import threading

import json
import numpy as np
import os

In [4]:
# Usamos los glove vectors 300D de wikipedia 2014. Por limitación de memoria no podemos usar un corpus más grande.
glove_file = datapath(os.getcwd() + '/glove.6B.300d.txt')
tmp_file = get_tmpfile(os.getcwd() + "/test_word2vec.txt")
print("Running script")
glove2word2vec(glove_file, tmp_file)
print("Loading Glove Vectors")
embedder = KeyedVectors.load_word2vec_format(tmp_file)

Running script
Loading Glove Vectors


In [5]:
# Funciones para metodo Embedding layer (se desecho)
# retorna secuencia de enteros a partir de secuencia de palabras.
def text_to_sequence(text_seq, word_index):
    result = []
    for word in text_seq:
        if word in word_index:
            result.append(word_index[word])
        else:
            word_index[word] = len(word_index)
            result.append(word_index[word])
    return np.array(result)

# Construimos index de palabras a medida que aparecen en el dataset. 
# Construimos matrices de enteros a partir del dataset.
def gen_data(dataset,word_index):
    contexts = []
    questions = []
    output = []
    for document in dataset:
        for paragraph in document['paragraphs']:
            for question in paragraph['qas']:
                # Tomamos la primera respuesta para generar el dataset final
                answer = question['answers'][0]
                # Pasamos secuencias a secuencias de enteros.
                contexts.append(text_to_sequence(paragraph['context_tokenized'],word_index))
                questions.append(text_to_sequence(question['question_tokenized'],word_index))
                #guardamos tupla de inicio y fin para el output.
                output.append((answer['answer_word_start'],answer['answer_word_end']))
    return contexts,questions,output,word_index



# Funciones para la keras sequence.

# Pasamos una secuencia de texto a un tensor de tamaño fijo.
# Quedan en 0 el padding y las palabras que no estan en GLove
def text_to_tensor(text_seq, word_vectors,sequence_length):
    result = np.zeros((sequence_length,300))
    for i,t in enumerate(text_seq):
        if t in word_vectors:
            result[i]  = word_vectors[t]
    return result

def data_counter(dataset):
    count = 0
    for document in dataset:
        for paragraph in document['paragraphs']:
            for question in paragraph['qas']:
                count +=1
    return count

# Secuencia para hacer multijob data generation
class TensorSequence(Sequence):

    def __init__(self, dataset,batch_size,word_vectors,context_length,question_length):
        self.dataset = []
        self.batch_size = batch_size
        self.data_count = data_counter(dataset)
        self.word_vectors = word_vectors
        self.context_length = context_length
        self.question_length = question_length
        print("Loading sequence")
        # Guardamos el dataset en formato tabla para poder indexar por batch size.
        for document in dataset:
            for paragraph in document['paragraphs']:
                for question in paragraph['qas']:
                    # Tomamos la primera respuesta para generar el dataset final
                    answer = question['answers'][0]
                    self.dataset.append([paragraph['context_tokenized'],question['question_tokenized'],answer['answer_word_start'],answer['answer_word_end']])

    # steps per batch
    def __len__(self):
        return self.data_count//self.batch_size

    
    
    # retorna un batch de tensores.
    def __getitem__(self, idx):
        contexts = None
        questions = None
        output_start = []
        output_end = []
        #iteramos sobre el batch pedido.
        for row in self.dataset[idx * self.batch_size:(idx + 1) * self.batch_size]:
            
            # pasamos sequencias de palabras a tensores
            if contexts is None:
                contexts = text_to_tensor(row[0],self.word_vectors,self.context_length)
            else:
                contexts = np.dstack((contexts,text_to_tensor(row[0],self.word_vectors,self.context_length)))
            if  questions is None:
                questions = text_to_tensor(row[1],self.word_vectors,self.question_length)
            else:
                questions= np.dstack((questions,text_to_tensor(row[1],self.word_vectors,self.question_length)))

            #guardamos tupla de inicio y fin para el output.
            output_start.append(row[2])
            output_end.append(row[3])
        return [np.moveaxis(contexts,2,0),np.moveaxis(questions,2,0)],[to_categorical(np.array(output_start),num_classes=self.context_length),to_categorical(np.array(output_end),num_classes=self.context_length)]
    

            
            
        




In [47]:

# generamos las secuencias de enteros para inyectar en el modelo de Keras.
with open("train-v1.1-pr.json", "r") as data:
    train = json.load(data)
with open("dev-v1.1-pr.json", "r") as data:
    test = json.load(data)

In [7]:
# Calculamos el tamaño máximo
def max_context(document):
    return max(document['paragraphs'], key= lambda x: len(x['context_tokenized']))

def max_question_par(paragraph):
    return max(paragraph['qas'], key= lambda x: len(x['question_tokenized']))

def max_paragraph(document):
    return max(document['paragraphs'], key=lambda x: len(max_question_par(x)['question_tokenized']))

MAX_CONTEXT = len(max(map(lambda x: max_context(x), train), key=lambda x:len(x['context_tokenized']))['context_tokenized'])
MAX_QUESTIONS = len(max_question_par(max_paragraph(max(train, key= lambda x:  len(max_question_par(max_paragraph(x))['question_tokenized']))))['question_tokenized'])
TRAIN_COUNT = data_counter(train)

# Diagrama del modelo

El modelo ocupado está inspirado en el trabajo de Raiman y Miller, "Globally Normalized Reader" (2017). Los pesos del modelo fueron omitidos en el diagrama por temas de visualización. Este modelo cuenta con dos etapas iniciales que se procesan por separado:

*Nota*: el subíndice $T$ podría ser diferente entre la codificación de la pregunta, el contexto o el input de la mezcla de ambos. 

- **Question Input**: la pregunta, luego de ser codificada usando GloVe, pasa por tres capas LSTM Bidireccionales. En el diagrama, el rectángulo gris que rodea cada unidad (representadas con la letra $h$, en los círculos de color azul) de las capas bidireccionales, representa una concatenación (tal como lo hace el modelo clásico de RNN bidireccional). El output de la capa Bidireccional final pasa por una capa densa con activación ReLu. La separación de la capa densa por unidades relacionadas con cada unidad de la última LSTM Bidireccional y su posterior concatenación, es para hacer explícito el procedimiento que hace keras por detrás. En el código esto no se ve reflejado explícitamente.

- **Context Input**: el contexto, luego de ser debidamente codificado con GloVe, pasa por solo una capa LSTM bidireccional para no aumentar demasiado el tamaño del modelo. El output de esta capa es concatenado con el output de la pregunta.

La siguiente etapa incluye la información tanto de la pregunta como del contexto. Esta pasa por dos capas LSTM Bidireccionales.



En el siguiente paso se generan dos outputs:

1) Output de la última unidad de la última capa bidireccional, el cual va hacia una capa densa con activación softmax.

2) Outputs de todas las unidades de la última capa bidireccional, los cuales son concatenados.

A partir de la primera softmax se obtiene la distribución de probabilidad el índice de la primera palabra de la respuesta, que esta codificado mediante un vector `one-hot`.

Luego, se hace una multiplicación _element-wise_ entre los outputs descritos (el primer output es concatenado con si mismo para que calcen las dimensiones en la multiplicación), de esta manera se condiciona lo que viene de aquí en adelante, por la probabilidad de la palabra inicial de la respuesta.

Finalmente, este resultado pasa por una última capa densa con activación softmax para realizar la predicción del índice final de la respuesta que esta codificado de forma análoga.

La arquitectura posee como hiperparametros la dimensión de la capa oculta de LSTMs para las preguntas y para el contexto.

La red posee 7M de parametros y por lo tanto una capacidad considerable. A modo de regularización las LSTMs tienen dropout con probabilidad 0.3

No hubo tiempo para experimentar mucho puesto que el modelo se demora bastante en entrenar. Se probaron 2 arquitecturas similares pero con más capas, que resultaban en modelos demasiado grandes. También se intento usar una capa de `Masking` para los 0 (palabras no encontradas o padding), pero al menos en los cortos experimentos que hicimos parecía ralentizar el entrenamiento.


## Declaración del Modelo

In [9]:
# Importamos dependencias
from keras.layers import Embedding, Input, Concatenate,Bidirectional,LSTM, Dense, Reshape, TimeDistributed, Activation, Flatten,Multiply,Conv1D, Masking
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
import tensorflow as tf
from keras import backend as K

# Para elegir GPU o multicore
num_cores = 4
CPU= False
GPU= not CPU
if GPU:
    num_GPU = 1
    num_CPU = 1
if CPU:
    num_CPU = 1
    num_GPU = 0

config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
        inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
session = tf.Session(config=config)
K.set_session(session)


In [ ]:
#MODEL PARAMS

CONTEXT_RECURRENT_DIM=100
QUESTION_RECURRENT_DIM=40

# Input de contexto 
context_input = Input(shape=(MAX_CONTEXT,300))

#Input de preguntas
question_input = Input(shape=(MAX_QUESTIONS,300))


# 3 capas de BiLSTM para las preguntas
question_nw = Bidirectional(LSTM(units=QUESTION_RECURRENT_DIM,dropout=0.3,return_sequences=True))(question_input)
question_nw = Bidirectional(LSTM(units=QUESTION_RECURRENT_DIM,dropout=0.3,return_sequences=True))(question_nw)
question_nw = Bidirectional(LSTM(units=QUESTION_RECURRENT_DIM,dropout=0.3,return_sequences=True))(question_nw)

# generamos vectores concatenables a los del contexto mediante capa densa.
question_nw = Dense(units=2*CONTEXT_RECURRENT_DIM, activation='relu')(question_nw)

# Codificacion inicial del contexto
context_nw = Bidirectional(LSTM(units=CONTEXT_RECURRENT_DIM,dropout=0.3,return_sequences=True,))(context_input)

#Concatenamos vectores de las preguntas a los vectores de las respuestas
result = Concatenate(axis=1)([context_nw,question_nw])

# Agregamos dos capas de BiLSTM
doc_encoding = Bidirectional(LSTM(units=CONTEXT_RECURRENT_DIM,dropout=0.3,return_sequences=True))(result)
doc_encoding = Bidirectional(LSTM(units=MAX_CONTEXT,dropout=0.3))(doc_encoding)

doc_dense = Dense(MAX_CONTEXT)(doc_encoding)
result_start = Activation(activation='softmax')(doc_dense)

# Ahora hacemos un producto elementwise entre la softmax y los vectores de obtenidos de cada palabra
# Asi condicionamos la palabra final por la distribución de la palabra inicial.

result_prod = Concatenate(axis=1)([result_start,result_start])
result_end = Multiply()([result_prod,doc_encoding])

# Capa Densa y activacion final
result_end = Dense(MAX_CONTEXT)(result_end)
result_end = Activation(activation='softmax')(result_end)

model = Model(inputs=[context_input,question_input],outputs=[result_start,result_end])

In [ ]:
model.summary()

## Entrenamiento

Se entrenó en una máquina con 8Gb de RAM, 4 Cores de CPU y una GTX1050M de 4Gb.

Se utilizo `Adam` con `batch_size` de `32`, `learning_rate` `0.001` y categorical cross entropy como función de pérdida, sumando las pérdidas de la palabra inicial y la palabra final de la respuesta.

Al entrenar no se paso un set de validación puesto que la memoria era un limitante fuerte y simplemente no había espacio. Es más se redujo el tamaño de la cola de batches a 5.

Finalmente aprovechando que la generación de batches se puede paralelizar mediante una `keras.utils.Sequence`, se usan 3 workers para alimentar el entrenamiento.

Luego de mucho optimizar e iterar se logró bajar el tiempo de entrenamiento a 3.5 horas por época,  lo que se mantuvo por alrededor de 16 horas llegando a 5 épocas, con un training accuracy de 4% y 2% para las palabras inicial y final.

Luego se paro el entrenamiento y se volvió a iniciar esta véz con un learning rate mas alto (`0.003`), lo cual funcionó bien inicialmente hasta que en algun momento el gradiente explotó y el accuracy bajo estrepitosamente. 

In [ ]:
# Parametros
BATCH_SIZE=32
EPOCHS=50
OPTIMIZER='adam'
LOSS= 'categorical_crossentropy'
generator= TensorSequence(train,BATCH_SIZE,embedder,MAX_CONTEXT,MAX_QUESTIONS)

checkpoint = ModelCheckpoint(filepath='weights.hdf5',monitor="loss", verbose=1)
callbacks_list = [checkpoint]

In [ ]:
model.compile(optimizer=OPTIMIZER,loss=LOSS, metrics=['accuracy'])
model.fit_generator(generator, steps_per_epoch = TRAIN_COUNT//BATCH_SIZE, max_queue_size=5, epochs = EPOCHS, verbose=1, callbacks=callbacks_list, use_multiprocessing=True, workers=3)

In [10]:
#entrenamos desde archivo guardado

model = load_model('weights.hdf5')
BATCH_SIZE=32
EPOCHS=50
OPTIMIZER= Adam(lr=0.003) #nuevo learning rate
LOSS= 'categorical_crossentropy'
generator= TensorSequence(train,BATCH_SIZE,embedder,MAX_CONTEXT,MAX_QUESTIONS)

checkpoint = ModelCheckpoint(filepath='weights.hdf5',monitor="loss", verbose=1)
callbacks_list = [checkpoint]

model.summary()

Loading sequence
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 40, 300)      0                                            
__________________________________________________________________________________________________
bidirectional_1 (Bidirectional) (None, 40, 80)       109120      input_2[0][0]                    
__________________________________________________________________________________________________
bidirectional_2 (Bidirectional) (None, 40, 80)       38720       bidirectional_1[0][0]            
__________________________________________________________________________________________________
input_1 (InputLayer)            (None, 677, 300)     0                                            
____________________________________________________________________________________________

In [ ]:
model.compile(optimizer=OPTIMIZER,loss=LOSS, metrics=['accuracy'])
model.fit_generator(generator, steps_per_epoch = TRAIN_COUNT//BATCH_SIZE, max_queue_size=5, epochs = EPOCHS, verbose=1, callbacks=callbacks_list, use_multiprocessing=True, workers=3)

Epoch 1/50
2737/2737 [==============================] - 13657s 5s/step - loss: 9.9162 - activation_1_loss: 4.9658 - activation_2_loss: 4.9504 - activation_1_acc: 0.0341 - activation_2_acc: 0.0154

Epoch 00001: saving model to weights.hdf5
Epoch 2/50
2737/2737 [==============================] - 13703s 5s/step - loss: 15.9773 - activation_1_loss: 11.0190 - activation_2_loss: 4.9583 - activation_1_acc: 0.0171 - activation_2_acc: 0.0150

Epoch 00002: saving model to weights.hdf5
Epoch 3/50
2522/2737 [==========================>...] - ETA: 17:54 - loss: 21.0849 - activation_1_loss: 16.1161 - activation_2_loss: 4.9689 - activation_1_acc: 9.9128e-05 - activation_2_acc: 0.0149

 ## Evaluación

Si bien no se alcanzó a hacer un entrenamiento apropiado de todas formas se intentará evaluar el modelo.

Por tiempo generaremos arreglo con indices originales y arreglo on indices predecidos y usaremos el metodo de sklearn para calcular el fscore.

In [70]:
# generador de test data.
test_generator = TensorSequence(test,BATCH_SIZE,embedder,MAX_CONTEXT,MAX_QUESTIONS)

Loading sequence


In [ ]:
y_true = []
y_pred = []
j = 0

# Predecimos por batch
for k in range(len(test_generator)):
    print(k,len(test_generator))
    batch = test_generator[k]
    for i in range(len(batch[1][0])):
        # agregamos la tupla real de inicio y fin.
        y_true.append((np.argmax(batch[1][0][i]),np.argmax(batch[1][1][i])))
    predict = model.predict_on_batch(batch[0])
    for i in range(len(predict[1])):
        #agregamos la tupla predecida.
        y_pred.append((np.argmax(predict[0][i]),np.argmax(predict[1][i])))
    j+=1


In [82]:
from sklearn.metrics import f1_score

# Mapeamos a arreglos 1 dimensionales para insertar en la función de sklearn.

y_true_start = list(map(lambda x: x[0], y_true))
y_pred_start = list(map(lambda x: x[0], y_pred))

y_true_end = list(map(lambda x: x[1], y_true))
y_pred_end = list(map(lambda x: x[1], y_pred))

y_true_1d = list(map(lambda x: "{0} {1}".format(*x), y_true))
y_pred_1d = list(map(lambda x: "{0} {1}".format(*x), y_pred))

In [81]:
print("f1 score inicio respuesta: {}".format(f1_score(y_true_start, y_pred_start, average='micro')*100))
print("f1 score fin respuesta: {}".format(f1_score(y_true_end, y_pred_end, average='micro')*100))
print("f1 score concatenacion: {}".format(f1_score(y_true_1d, y_pred_1d, average='micro')*100))

f1 score inicio respuesta: 2.9356060606060606
f1 score fin respuesta: 1.3636363636363635
f1 score concatenacion: 0.9090909090909091
